In [1]:
%load_ext autoreload
%autoreload 2

from api_tricount import *


In [2]:

data, dict = fetch_data()
data = build_data(data)
detail = split_data(data, dict)
postes, result = concat_data(detail, dict)



    # --------------------------------- #
    # Connection to Google Sheet        #
    # and extract Spreadsheets          #
    #  - exported tricount data         #
    #  - category dictonnary            #
    # --------------------------------- #
    
 -- data fetched
(1023, 17)
Index(['Unnamed: 0', 'Titre', 'Montant', 'Devise', 'Taux de change',
       'Montant dans la devise du tricount (EUR)', 'Type de transaction',
       'Catégorie', 'Payé par', 'Payé par Commun', 'Payé par Lucie',
       'Payé par Vincent', 'Impacté à Commun', 'Impacté à Lucie',
       'Impacté à Vincent', 'Date & heure', 'URL des images'],
      dtype='object')
 -- dict fetched 
{'Alimentation': 'Quotidien', 'Shopping': 'Achats', 'Logement': 'Achats', 'Loyer & charges': 'Quotidien', 'Divertissement': 'Extra', 'Sport': 'Loisir', 'Restaurant & bar': 'Extra', 'Investissement': 'Investissement', 'Transport': 'Quotidien', 'Week-end': 'Loisir', 'Soin de santé': 'Quotidien', 'Livres': 'Achats', 'Activités': 'Lois

In [83]:
%load_ext autoreload
%autoreload 2
from api_portfolio import build_data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
data = build_data()
s = data.iloc[-1]


    # ----------------------------------------------
    # Get data, concate, merge and extract metrics
    # for portfolio Dashboard 
    # ----------------------------------------------
    
              Market Currency     Forex IsDepot  PriceFmt AmountFmt  ValueFmt
Asset                                                                        
€           EUREUR=X        €  EUREUR=X    TRUE  {:.0f} €    {:.0f}  {:.0f} €
$              EUR=X        €  EUREUR=X    TRUE  {:.4f} €    {:.0f}  {:.0f} €
USDT           EUR=X        €  EUREUR=X    TRUE  {:.4f} €    {:.0f}  {:.0f} €
ESE           ESE.PA        €  EUREUR=X          {:.2f} €    {:.0f}  {:.2f} €
PUST                        €  EUREUR=X          {:.2f} €    {:.0f}  {:.2f} €
RS2K                        €  EUREUR=X          {:.2f} €    {:.0f}  {:.2f} €
OBLI         OBLI.PA        €  EUREUR=X          {:.2f} €    {:.0f}  {:.0f} €
GOLD         GOLD.PA        €  EUREUR=X          {:.2f} €    {:.0f}  {:.0f} €
BTC          BTC-USD     U

In [85]:
# s['PnL', 'ZEN', '$']
s[:, 'ZEN', '$'].index


Index(['Amount', 'Deposit', 'DepositEUR', 'Cotation', 'IsDepot', 'Currency',
       'PriceFmt', 'AmountFmt', 'ValueFmt'],
      dtype='object')

In [31]:
import streamlit as st
import gspread as gs 
import pandas as pd
import numpy as np
import yfinance as yf

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
# ss = gc.open_by_key(st.secrets['portfolio'].spreadsheet_key)
# dicts = pd.DataFrame(ss.worksheet('Dict').get_all_records())
# operation = pd.DataFrame(ss.worksheet('Operations').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
# greenbull = pd.DataFrame(ss.worksheet('GREENBULL').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
# assets = dicts.set_index('Asset')
# print(assets, '\n')

# market = yf.download(' '.join(list(assets['Forex'])+list(assets['Market'])[:-1]), start='2021-04-28')['Close']
# market = pd.concat([market, greenbull], axis=1)
# market = pd.concat([market], keys=['Market'], axis=1)
# market = pd.concat([market], keys=['Cotation'], axis=1)

operation = operation
assets = assets
market = market

df = market.ffill().bfill()
df['DepositEUR', 'All', 'All'] = 0
df['InvestedEUR', 'All', 'All'] = 0
df['ValueEUR', 'All', 'All'] = 0
df['PnLEUR', 'All', 'All'] = 0
df['CashEUR', 'All', 'All'] = 0

for portfolio in np.unique(operation['Portfolio']):
    dfp: pd.DataFrame = operation[operation['Portfolio'] == portfolio]
    print(portfolio)

    for asset in np.unique(dfp['Asset']):
        dfa: pd.DataFrame = dfp[dfp['Asset'] == asset]
        dfa = dfa.groupby('Date').agg({'Quantity': "sum", 'Total': "sum"})

        qty, tot = pd.Series(dtype=float), pd.Series(dtype=float)
        for idx in dfa.index:
            qty[idx] = np.sum(dfa.loc[:idx]['Quantity'])
            tot[idx] = np.sum(dfa.loc[:idx]['Total'])

        df = pd.concat([df, pd.DataFrame({
            ('Quantity', portfolio, asset): qty,
            ('Total', portfolio, asset): tot,
        })], axis=1).ffill().fillna(0)

        print(f"\t{assets.loc[asset]['Class']}:\t{asset}")
        toEUR = df['Cotation', 'Market', assets.loc[asset]['Forex']]
        df['Cotation', portfolio, asset] = df['Cotation', 'Market', assets.loc[asset]['Market']]
        df['Class', portfolio, asset] = assets.loc[asset]['Class']
        df['Currency', portfolio, asset] = assets.loc[asset]['Currency']
        df['PriceFmt', portfolio, asset] = assets.loc[asset]['PriceFmt']
        df['AmountFmt', portfolio, asset] = assets.loc[asset]['AmountFmt']
        df['ValueFmt', portfolio, asset] = assets.loc[asset]['ValueFmt']

        # if assets.loc[asset]['Class'] == "Cash":
        #     df['Deposit', portfolio, asset] = df['Total', portfolio, asset]

        # else:
        df['Amount', portfolio, asset] = df['Quantity', portfolio, asset]
        df['Invested', portfolio, asset] = df['Total', portfolio, asset]

        df['PRU', portfolio, asset] = df['Invested', portfolio, asset] / df['Amount', portfolio, asset]
        df['Value', portfolio, asset] = df['Amount', portfolio, asset] * df['Cotation', portfolio, asset]
        df['PnL', portfolio, asset] = df['Value', portfolio, asset] - df['Invested', portfolio, asset]
        df['InvestedEUR', portfolio, asset] = df['Invested', portfolio, asset] / toEUR
        df['ValueEUR', portfolio, asset] = df['Value', portfolio, asset] / toEUR
        df['PnLEUR', portfolio, asset] = df['PnL', portfolio, asset] / toEUR

df.loc[:, pd.IndexSlice[:,'ZEN',['BTC']]] 

DMA
	Cash:	€
ZEN
	Cash:	$
	Crypto:	BTC
	Stocks:	ESE
	Commo:	GOLD
	Stocks:	GREENBULL
	Bonds:	OBLI
	Cash:	USDT
	Cash:	€


,Quantity,Total,Cotation,Class,Currency,PriceFmt,AmountFmt,ValueFmt,Amount,Invested,PRU,Value,PnL,InvestedEUR,ValueEUR,PnLEUR
,ZEN,ZEN,ZEN,ZEN,ZEN,ZEN,ZEN,ZEN,ZEN,ZEN,ZEN,ZEN,ZEN,ZEN,ZEN,ZEN
,BTC,BTC,BTC,BTC,BTC,BTC,BTC,BTC,BTC,BTC,BTC,BTC,BTC,BTC,BTC,BTC
2021-04-01,0.00000,0.00,54824.703125,Crypto,USDT,{:.0f} $,{:.4f},{:.0f} $,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2021-04-28,0.00000,0.00,54824.703125,Crypto,USDT,{:.0f} $,{:.4f},{:.0f} $,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2021-04-29,0.00000,0.00,53555.109375,Crypto,USDT,{:.0f} $,{:.4f},{:.0f} $,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2021-04-30,0.00000,0.00,57750.175781,Crypto,USDT,{:.0f} $,{:.4f},{:.0f} $,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2021-05-01,0.00000,0.00,57828.050781,Crypto,USDT,{:.0f} $,{:.4f},{:.0f} $,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-10,0.05524,1539.84,21651.183594,Crypto,USDT,{:.0f} $,{:.4f},{:.0f} $,0.05524,1539.84,27875.452571,1196.011382,-343.828618,1433.437056,1113.366995,-320.070061
2023-02-11,0.05524,1539.84,21870.875000,Crypto,USDT,{:.0f} $,{:.4f},{:.0f} $,0.05524,1539.84,27875.452571,1208.147135,-331.692865,1433.437056,1124.664168,-308.772888
